<a href="https://colab.research.google.com/github/francoishcm/BackTesting/blob/master/Zipline_Reloaded_BacktestEngine_6_05072022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧰**INSTALL PACKAGES**

In [ ]:
# Mount Google Drive 
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install ta-lib v0.4.0
%%bash
wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
tar -xzf ta-lib-0.4.0-src.tar.gz
cd ta-lib/
./configure
make
make install

In [ ]:
# Install zipline
%pip install zipline-reloaded

In [ ]:
# Install Pyfolio
!pip install pyfolio-reloaded

In [ ]:
# Install matplot library
!pip install matplotlib

In [ ]:
# Install data bundle 'Quandl'
!pip install quandl

# 💽**SET WORKING DIRECTORY**

In [ ]:
import os 

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/content/drive/"


In [ ]:
import os 

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!


# choose where you want your project files to be saved
project_folder = "MyDrive/Colab Notebooks/My Project Folder"


In [ ]:
def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  !touch 'new_file_in_working_directory.txt'
  print('\nYour working directory was changed to ' + root_dir + project_folder + \
        "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

create_and_set_working_directory(project_folder)

In [ ]:
# Confirm current working directory
!pwd

In [ ]:
import os
for f in os.listdir("//content/drive/MyDrive/Colab Notebooks/My Project Folder"):
	print(f)

# ⏳**REGISTER & INGEST DATA**

In [3]:
# Ingest custom bundle
!zipline ingest --bundle 'crypto'

[2022-07-05 08:59:01.386905] INFO: zipline.data.bundles.core: Ingesting crypto.
 | BITCOIN: sid 0


In [4]:
# Confirm existing bundles
!zipline bundles

crypto <no ingestions>
csvdir <no ingestions>
quandl <no ingestions>
quantopian-quandl <no ingestions>
random_futures_data <no ingestions>
random_stock_data <no ingestions>


In [1]:
#@title
# clean everything older than <date>
!zipline clean -b crypto --after 2022-04-13

[2022-07-05 08:58:34.241725] INFO: zipline.data.bundles.core: Cleaning 2022-07-05T08;56;16.176391.
[2022-07-05 08:58:34.242064] INFO: zipline.data.bundles.core: Cleaning 2022-07-05T08;58;03.920921.


In [ ]:
#@title
'''
INGEST QUANDL STOCK BUNDLE
'''

#Suppress Warning messages
import warnings
warnings.filterwarnings('ignore')

# Ingest bundle API
!QUANDL_API_KEY=KUnssHvVERHb5XYu9C1- zipline ingest -b 'quandl'

In [2]:
# Confirm existing bundle that were ingested
!zipline bundles

crypto <no ingestions>
csvdir <no ingestions>
quandl <no ingestions>
quantopian-quandl <no ingestions>
random_futures_data <no ingestions>
random_stock_data <no ingestions>


# 📊**ANDREAS CLENOW MODELS**

## Clenow Momentum Model

In [ ]:
#@title
%matplotlib inline

import zipline
from zipline import run_algorithm
from zipline.api import order_target_percent, symbol, set_commission, set_slippage, schedule_function, date_rules, time_rules
from pandas import Timestamp
from datetime import datetime
import pytz
import matplotlib.pyplot as plt 
import pyfolio as pf
import pandas as pd 
import numpy as np 
from scipy import stats
from zipline.finance.commission import PerDollar
from zipline.finance.slippage import VolumeShareSlippage, FixedSlippage

#Model Settings

intial_portfolio = 100000
momentum_window = 125
minimum_momentum = 40
portfolio_size = 30
vola_window = 20

#Commission and Slippage Settings

enable_commission = True 
commission_pct = 0.001 
enable_slippage = True 
slippage_volume_limit = 0.025
slippage_impact = 0.05

def momentum_score(ts):

  #Input: Price time series.Output: Annualized exponential regression slope, multiplied by the R2

  # Make a list of consecutive numbers 
  x = np.arange(len(ts))
  # Get logs
  log_ts = np.log(ts)
  # Calculate regression values
  slope, intercept, r_value, p_value, std_err = stats.linregress(x, log_ts) 
  # Annualize percent
  annualized_slope = (np.power(np.exp(slope), 252) - 1) * 100 
  #Adjust for fitness
  score = annualized_slope * (r_value ** 2) 
  return score

def volatility(ts):
  return ts.pct_change().rolling(vola_window).std().iloc[-1]

def output_progress(context):

  #Output some performance numbers during backtest run 
  #This code just prints out the past month's performance,
  # so that we have something to look at while the backtest runs.

  # Get today's date
  today = zipline.api.get_datetime().date()

  # Calculate percent difference since last month
  perf_pct = (context.portfolio.portfolio_value / context.last_month) - 1

  # Print performance, format as percent with two decimals. 
  print("{} - Last Month Result: {:.2%}".format(today, perf_pct))

  # Remember today's portfolio value for next month's calculation 
  context.last_month = context.portfolio.portfolio_value

#Initialization and trading logic


def initialize(context):

  # Set commission and slippage.
  if enable_commission:
    comm_model = PerDollar(cost=commission_pct) 
  else:
    comm_model = PerDollar(cost=0.0) 
  set_commission(comm_model)
  if enable_slippage: slippage_model=VolumeShareSlippage(volume_limit=slippage_volume_limit,
price_impact=slippage_impact) 
  else:
    slippage_model=FixedSlippage(spread=0.0) 
  set_slippage(slippage_model)

  # Used only for progress output. 
  context.last_month = intial_portfolio

  # Store index membership
  context.index_members = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/index members/sp500.csv', engine='python', error_bad_lines=False)


  #Schedule rebalance monthly. 
  schedule_function(
    func=rebalance, 
    date_rule=date_rules.month_start(), 
    time_rule=time_rules.market_open()
  )

def rebalance(context, data):
  # Write some progress output during the backtest 
  output_progress(context)

  # First, get today's date
  today = zipline.api.get_datetime().date()

  # Second, get the index makeup for all days prior to today.
  all_prior = context.index_members.loc[context.index_members.index < today]

  todays_universe = [ 
    symbol(ticker) for ticker in
    context.index_members.loc[context.index_members.index < today].iloc[-1,0].split(',')
  ]

  # Get historical data
  hist = data.history(todays_universe, "close", momentum_window, "1d")

  # Make momentum ranking table
  ranking_table = hist.apply(momentum_score).sort_values(ascending=False)

  #Sell Logic
  #First we check if any existing position should be sold.
    #Sell if stock is no longer part of index.
    #Sell if stock has too low momentum value.

  kept_positions = list(context.portfolio.positions.keys()) 
  for security in context.portfolio.positions:
    if (security not in todays_universe): 
      order_target_percent(security, 0.0) 
      kept_positions.remove(security)
    elif ranking_table[security] < minimum_momentum: 
      order_target_percent(security, 0.0) 
      kept_positions.remove(security)

  #Stock Selection Logic
  #Check how many stocks we are keeping from last month.
  #Fill from top of ranking list, until we reach the desired total number of portfolio holdings.

  replacement_stocks = portfolio_size - len(kept_positions) 
  buy_list = ranking_table.loc[
    ~ranking_table.index.isin(kept_positions)][:replacement_stocks]
  new_portfolio = pd.concat(
    (buy_list,
    ranking_table.loc[ranking_table.index.isin(kept_positions)])
  )
  buy_list = ranking_table.loc[
    ~ranking_table.index.isin(kept_positions)][:replacement_stocks]

  #Calculate inverse volatility for stocks, and make target position weights.

  vola_table = hist[new_portfolio.index].apply(volatility) 
  inv_vola_table = 1 / vola_table
  sum_inv_vola = np.sum(inv_vola_table) 
  vola_target_weights = inv_vola_table / sum_inv_vola
  for security, rank in new_portfolio.iteritems(): 
    weight = vola_target_weights[security]
    if security in kept_positions:
      order_target_percent(security, weight)
    else:
      if ranking_table[security] > minimum_momentum: 
        order_target_percent(security, weight)

def analyze(context, perf):
  perf['max'] = perf.portfolio_value.cummax() 
  perf['dd'] = (perf.portfolio_value / perf['max']) - 1 
  maxdd = perf['dd'].min()

  ann_ret = (np.power((perf.portfolio_value.iloc[-1] / perf.portfolio_value.iloc[0]),(252 / len(perf)))) - 1

  print("Annualized Return: {:.2%} Max Drawdown: {:.2%}".format(ann_ret, maxdd))

  return
'''
start = datetime(1997, 1, 1, 8, 15, 12, 0, pytz.UTC)
end = datetime(2018, 12, 31, 8, 15, 12, 0, pytz.UTC)
'''

start = pd.Timestamp('1997-1-1', tz='utc')
end = pd.Timestamp('2018-12-31', tz='utc')

perf = zipline.run_algorithm(start=start, end=end, 
                             initialize=initialize, 
                             analyze=analyze, 
                             capital_base=intial_portfolio, 
                             data_frequency = 'daily', 
                             bundle='quandl' )

## Clenow Trend Model

In [ ]:

%matplotlib inline

import zipline
from zipline.api import future_symbol,  \
    set_commission, set_slippage, schedule_function, date_rules, \
    time_rules, continuous_future, order_target
from pandas import Timestamp
import pytz
import datetime as datetime
import matplotlib.pyplot as plt
import pyfolio as pf
import pandas as pd
import numpy as np  
from zipline.finance.commission import PerTrade, PerContract
from zipline.finance.slippage import VolumeShareSlippage, \
    FixedSlippage, VolatilityVolumeShare

# These lines are for the dynamic text reporting
from IPython.display import display
import ipywidgets as widgets
out = widgets.HTML()
display(out)

"""
Model Settings
"""
starting_portfolio = 50000000
risk_factor = 0.0015
stop_distance = 3
breakout_window = 50
vola_window = 40
slow_ma = 80
fast_ma = 40
enable_commission = True
enable_slippage = True  


def report_result(context, data):
    context.months += 1
    today = zipline.api.get_datetime().date()
    # Calculate annualized return so far
    ann_ret = np.power(context.portfolio.portfolio_value / starting_portfolio, 
                   12 / context.months) - 1
    
    # Update the text
    out.value = """{} We have traded <b>{}</b> months 
    and the annualized return is <b>{:.2%}</b>""".format(today, context.months, ann_ret)

def roll_futures(context, data):
    open_orders = zipline.api.get_open_orders()
    
    for held_contract in context.portfolio.positions:
        # don't roll positions that are set to change by core logic
        if held_contract in open_orders: 
            continue
        
        # Save some time by only checking rolls for
        # contracts stopping trading in the next days
        days_to_auto_close = (
            held_contract.auto_close_date.date() - data.current_session.date()
        ).days
        if days_to_auto_close > 5:
            continue        
        
        # Make a continuation
        continuation = continuous_future(
                held_contract.root_symbol, 
                offset=0, 
                roll='volume', 
                adjustment='mul'
                )
        
        # Get the current contract of the continuation
        continuation_contract = data.current(continuation, 'contract')
        
        if continuation_contract != held_contract:
            # Check how many contracts we hold
            pos_size = context.portfolio.positions[held_contract].amount         
            # Close current position
            order_target(held_contract, 0)
            # Open new position
            order_target(continuation_contract, pos_size)     
            
def position_size(portfolio_value, std, point_value):
    target_variation = portfolio_value * risk_factor
    contract_variation = std * point_value
    contracts = target_variation / contract_variation
    return int(np.nan_to_num(contracts)) 
    
def initialize(context):
    
    """
    Cost Settings
    """
    if enable_commission:
        comm_model = PerContract(cost=0.85, exchange_fee=1.5)
    else:
        comm_model = PerTrade(cost=0.0)
        
    set_commission(us_futures=comm_model)
    
    if enable_slippage:
        slippage_model=VolatilityVolumeShare(volume_limit=0.2)
    else:
        slippage_model=FixedSlippage(spread=0.0)      
        
    set_slippage(us_futures=slippage_model)
    
    """
    Markets to trade
    """ 
    currencies = [
        'AD',
        'BP',
        'CD',
        'CU',
        'DX',
        'JY',
        'NE',
        'SF',
    ]
    
    agricultural = [
        '_C',
        'CT',
        'FC',
        'KC',
        'LR',
        'LS',
        '_O',
        '_S',
        'SB',
        'SM',
        '_W',
    ]
    nonagricultural = [
        'CL',
        'GC',
        'HG',
        'HO',
        'LG',
        'NG',
        'PA',
        'PL',
        'RB',
        'SI',
    ]
    equities = [
        'ES',
        'NK',
        'NQ',
        'TW',
        'VX',
        'YM',
    ]
    rates = [
        'ED',
        'FV',
        'TU',
        'TY',
        'US',
    ]
    
    # Make a list of all the markets
    markets = currencies + agricultural + nonagricultural + equities + rates
    
    # Make a list of all continuations
    context.universe = [
        continuous_future(market, offset=0, roll='volume', adjustment='mul')
            for market in markets
    ]
    
    # We'll use these to keep track of best position reading
    # Used to calculate stop points.
    context.highest_in_position = {market: 0 for market in markets} 
    context.lowest_in_position = {market: 0 for market in markets}    
    
    # Schedule the daily trading
    schedule_function(daily_trade, date_rules.every_day(), time_rules.market_close())
    
    # We'll just use this for the progress output
    # during the backtest. Doesn't impact anything.
    context.months = 0    
    
    # Schedule monthly report output
    schedule_function(
        func=report_result,
        date_rule=date_rules.month_start(),
        time_rule=time_rules.market_open()
    ) 
    
def analyze(context, perf):
    returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(perf)
    pf.create_returns_tear_sheet(returns, benchmark_rets=None)
    
def daily_trade(context, data):
    # Get continuation data
    hist = data.history(
        context.universe, 
        fields=['close','volume'], 
        frequency='1d', 
        bar_count=250,
    )
    
    # Calculate trend
    hist['trend'] = hist['close'].ewm(span=fast_ma).mean() > hist['close'].ewm(span=slow_ma).mean()    
    
    # Make dictionary of open positions
    open_pos = {
        pos.root_symbol: pos 
        for pos in context.portfolio.positions
    } 
    
    # Iterate markets, check for trades
    for continuation in context.universe:
        
        # Get root symbol of continuation
        root = continuation.root_symbol
        
        # Slice off history for just this market
        h = hist.xs(continuation, 2)
        
        # Get standard deviation
        std = h.close.diff()[-vola_window:].std()

        if root in open_pos: # Position is open

            # Get position
            p = context.portfolio.positions[open_pos[root]]
            
            if p.amount > 0: # Position is long
                if context.highest_in_position[root] == 0: # First day holding the position
                    context.highest_in_position[root] = p.cost_basis
                else:
                    context.highest_in_position[root] = max(
                        h['close'].iloc[-1], context.highest_in_position[root]
                    ) 
                    
                # Calculate stop point
                stop = context.highest_in_position[root] - (std  * stop_distance)
                # Check if stop is hit
                if h.iloc[-1]['close'] < stop:
                    contract = open_pos[root]
                    order_target(contract, 0)
                    context.highest_in_position[root] = 0
                # Check if trend has flipped
                elif h['trend'].iloc[-1] == False:
                    contract = open_pos[root]
                    order_target(contract, 0)
                    context.highest_in_position[root] = 0
                    
            else: # Position is short
                if context.lowest_in_position[root] == 0: # First day holding the position
                    context.lowest_in_position[root] = p.cost_basis
                else:
                    context.lowest_in_position[root] = min(
                        h['close'].iloc[-1], context.lowest_in_position[root]
                    )
                
                # Calculate stop point
                stop = context.lowest_in_position[root] + (std  * stop_distance)
                
                # Check if stop is hit
                if h.iloc[-1]['close'] > stop:
                    contract = open_pos[root]
                    order_target(contract, 0)
                    context.lowest_in_position[root] = 0
                # Check if trend has flipped
                elif h['trend'].iloc[-1] == True:
                    contract = open_pos[root]
                    order_target(contract, 0)
                    context.lowest_in_position[root] = 0                         
        
        else: # No position on
            if h['trend'].iloc[-1]: # Bull trend
                # Check if we just made a new high
                if h['close'][-1] == h[-breakout_window:]['close'].max(): 
                    contract = data.current(continuation, 'contract')

                    contracts_to_trade = position_size( \
                                                       context.portfolio.portfolio_value, \
                                                       std, \
                                                       contract.price_multiplier)
                    
                    # Limit size to 20% of avg. daily volume
                    contracts_cap = int(h['volume'][-20:].mean() * 0.2)
                    contracts_to_trade = min(contracts_to_trade, contracts_cap)
                    
                    # Place the order
                    order_target(contract, contracts_to_trade)
             
            else: # Bear trend
                # Check if we just made a new low
                if h['close'][-1] == h[-breakout_window:]['close'].min(): 
                    contract = data.current(continuation, 'contract')

                    contracts_to_trade = position_size( \
                                                       context.portfolio.portfolio_value, \
                                                       std, \
                                                       contract.price_multiplier)
                    
                    # Limit size to 20% of avg. daily volume
                    contracts_cap = int(h['volume'][-20:].mean() * 0.2)
                    contracts_to_trade = min(contracts_to_trade, contracts_cap)
                    
                    # Place the order
                    order_target(contract, -1 * contracts_to_trade)
    
    # If we have open positions, check for rolls
    if len(open_pos) > 0:   
        roll_futures(context, data)                
                        

start = pd.Timestamp('2003-01-01', tz='utc')
end = pd.Timestamp('2017-12-31', tz='utc')

perf = zipline.run_algorithm(
    start=start, end=end, 
    initialize=initialize, 
    analyze=analyze,
    capital_base=starting_portfolio,  
    data_frequency = 'daily', 
    bundle='random_futures_data' ) 




# 💻**MODEL TESTING**

## My Tests

## DOW Jones Momentum Model

In [4]:

%matplotlib inline

import zipline
from zipline import run_algorithm
from zipline.api import order_target_percent, symbol, set_commission, set_slippage, schedule_function, date_rules, time_rules
import matplotlib.pyplot as plt 
import pyfolio as pf
import pandas as pd 
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
from zipline.finance.commission import PerDollar
from zipline.finance.slippage import VolumeShareSlippage, FixedSlippage


initial_portfolio = 10000
minimum_momentum = 10
portfolio_size = 1
vola_window = 30

def momentum_score(ts):

  #Input: Price time series.Output: Annualized exponential regression slope, multiplied by the R2

  # Make a list of consecutive numbers 
  x = np.arange(len(ts))
  # Get logs
  log_ts = np.log(ts)
  # Calculate regression values
  slope, intercept, r_value, p_value, std_err = stats.linregress(x, log_ts) 
  # Annualize percent
  annualized_slope = (np.power(np.exp(slope), 252) - 1) * 100 
  #Adjust for fitness
  score = annualized_slope * (r_value ** 2) 
  return score

def volatility(ts):
  return ts.pct_change().rolling(vola_window).std().iloc[-1]


#Initialization and trading logic


def initialize(context):

  context.rolling_window = 100

  #Commission and Slippage Settings

  enable_commission = True 
  commission_pct = 0.001 
  enable_slippage = True 
  slippage_volume_limit = 0.025
  slippage_impact = 0.05

  """
  dji = ["AAVE","ADA","ALPHA","BCH","BTC",
          "DOGE","DOT","ETH","LTC","USDT",
          "XLM","XMR","XRP"]
  """
  dji = ["BITCOIN"]

  
  # Make a list of symbols from the list of tickers
  context.dji_symbols = [symbol(s) for s in dji]

  # Set commission and slippage.
  if enable_commission:
    comm_model = PerDollar(cost=commission_pct) 
  else:
    comm_model = PerDollar(cost=0.0) 
  set_commission(comm_model)
  if enable_slippage: slippage_model=VolumeShareSlippage(volume_limit=slippage_volume_limit,
price_impact=slippage_impact) 
  else:
    slippage_model=FixedSlippage(spread=0.0) 
  set_slippage(slippage_model)


  #Schedule rebalance monthly. 
  schedule_function(
    func=rebalance, 
    date_rule=date_rules.month_start(), 
    time_rule=time_rules.market_open()
  )

def rebalance(context, data):

  # Get historical data
  hist = data.history(context.dji_symbols, "close", context.rolling_window, "1d")

  # Make momentum ranking table
  ranking_table = hist.apply(momentum_score).sort_values(ascending=False)

  #Sell Logic
  #First we check if any existing position should be sold.
    #Sell if stock is no longer part of index.
    #Sell if stock has too low momentum value.

  kept_positions = list(context.portfolio.positions.keys()) 
  for security in context.portfolio.positions:
    if ranking_table[security] < minimum_momentum: 
      order_target_percent(security, 0.0)
      kept_positions.remove(security)
 
  #Stock Selection Logic
  #Check how many stocks we are keeping from last month.
  #Fill from top of ranking list, until we reach the desired total number of portfolio holdings.

  replacement_stocks = portfolio_size - len(kept_positions) 
  buy_list = ranking_table.loc[
    ~ranking_table.index.isin(kept_positions)][:replacement_stocks]
  new_portfolio = pd.concat(
    (buy_list,
    ranking_table.loc[ranking_table.index.isin(kept_positions)])
  )
  buy_list = ranking_table.loc[
    ~ranking_table.index.isin(kept_positions)][:replacement_stocks]

  #Calculate inverse volatility for stocks, and make target position weights.

  vola_table = hist[new_portfolio.index].apply(volatility) 
  inv_vola_table = 1 / vola_table
  sum_inv_vola = np.sum(inv_vola_table) 
  vola_target_weights = inv_vola_table / sum_inv_vola
  for security, rank in new_portfolio.iteritems(): 
    weight = vola_target_weights[security]
    if security in kept_positions:
      order_target_percent(security, weight)
    else:
      if ranking_table[security] > minimum_momentum: 
        order_target_percent(security, weight)


def analyze(context, perf):
    
  # Use PyFolio to generate a performance report
  returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(perf)

  benchmark_period_return = perf['benchmark_period_return']

  daily_benchmark_returns = np.exp(np.log(benchmark_period_return + 1.0).diff()) - 1

  # Create tear sheet
  pf.create_full_tear_sheet(returns, positions=positions, transactions=transactions, benchmark_rets=None)

start= pd.Timestamp('2011-02-01', tz='utc')
end = pd.Timestamp('2021-12-31', tz='utc')

perf = zipline.run_algorithm(start=start, 
                             end=end, 
                             initialize=initialize, 
                             analyze=analyze, 
                             capital_base=initial_portfolio, 
                             data_frequency='daily',
                             trading_calendar='NYSE', 
                             bundle='crypto')

AttributeError: ignored

In [ ]:
#Export Performance Result to disk in csv file

perf.portfolio_value.to_csv('dji_momentum_model.csv')

## Equal Weight Momementum Model

In [ ]:
# This ensures that our graphs will be shown properly in the notebook.
%matplotlib inline

# Import a few libraries we need
from zipline import run_algorithm
from zipline.api import order_target_percent, record, symbol, set_benchmark
import pyfolio as pf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def initialize(context):

  # Which stock to trade
  dji = ["AAPL","AXP",
         "CAT", "CSCO",
         "CVX", "TSLA",
         "DIS","DWDP",
         "GS","HD",
         "IBM","INTC",
         "JNJ","JPM",
         "KO","MCD",
         "MMM","MRK",
         "MSFT","NKE",
         "PFE","PG",
         "TRV","UNH",
         "UTX","V",
         "VZ","WBA",
         "WMT","XOM"]

  # Make a list of symbols from the list of tickers
  context.dji_symbols = [symbol(s) for s in dji]

  # Moving average window
  context.index_average_window = 100
  
def handle_data(context, data):
  
  # Get history for all the stocks
  stock_hist = data.history(context.dji_symbols, "close", context.index_average_window, "1d")

  # Make an empty DataFrame to start with
  stock_analytics = pd.DataFrame()

  # Add column for above or below average
  stock_analytics['above_mean'] = stock_hist.iloc[-1] > stock_hist.mean()

  # Set weight for stocks to buy
  stock_analytics.loc[stock_analytics['above_mean'] == True, 'weight'] = 1/len(context.dji_symbols)

  # Set weight to zero for the rest
  stock_analytics.loc[stock_analytics['above_mean'] == False, 'weight'] = 0.0

  # Iterate each row and place trades
  for stock, analytics in stock_analytics.iterrows():

    # Check if the stock can be traded
    if data.can_trade(stock):

      # Place the trade
      order_target_percent(stock, analytics['weight'])

def analyze(context, perf):

  # Use PyFolio to generate a performance report
  returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(perf)

  benchmark_period_return = perf['benchmark_period_return']

  daily_benchmark_returns = np.exp(np.log(benchmark_period_return + 1.0).diff()) - 1

  # Create tear sheet
  pf.create_full_tear_sheet(returns, positions=positions, transactions=transactions, benchmark_rets=None)

# Set start and end date
start_date = pd.Timestamp('2003-1-1', tz='utc')
end_date = pd.Timestamp('2017-12-31', tz='utc')

# Fire off the backtest
perf = run_algorithm(
start=start_date,
end=end_date,
initialize=initialize,
analyze=analyze,
handle_data=handle_data,
capital_base=10000,
data_frequency = 'daily', 
bundle= 'quandl',)

In [ ]:
#Export performance results to disk in csv file

perf.portfolio_value.to_csv('ewm_momentum_model.csv')

## Compare Performance Results to Bencmark

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import empyrical as em
from IPython.core.display import display, HTML
import pandas as pd
import numpy

base_path = '/content/drive/MyDrive/Colab Notebooks/My Project Folder/Backtests/'

benchmark_name = {"SPXTR" : "S&P 500 Total Return Index"}
bm = "SPXTR"
bm_name = benchmark_name[bm]

strat_names = {
    "dji_momentum_model" : "Dow Jones 30 Momentum Strategy",
    "core_trend" : "Core Trend Strategy",    
    "time_return" : "Time Return Strategy",
    "counter_trend" : "Counter Trend Strategy",
    "curve_trading" : "Curve Trading Strategy",
    "equity_momentum" : "Equity Momentum Strategy",
    "dow_model" : "Dow_Strategy",
    "EWM_momentum_model" : "Equal Weight Momentum Strategy"
}


strat = 'dji_momentum_model' #change line of code to desired model name of strategy
strat_name = strat_names[strat]

df = pd.read_csv(base_path + strat + '.csv', index_col=0, parse_dates=True, names=[strat] )
'''
df[bm_name] = pd.read_csv(base_path + bm + '.csv', index_col=0,parse_dates=True, names=[bm])
'''


df.index = df.index.strftime('%d/%m/%Y')


bm_name = pd.read_csv(base_path + bm + '.csv', index_col=0,parse_dates=True, names=[bm])
bm_name
bm_name.index = bm_name.index.strftime('%d/%m/%Y')

new_df = pd.concat([df, bm_name], axis=1, join='inner')

#change line of code to desired model name of strategy
new_df['dji_momentum_model'] = new_df['dji_momentum_model'].astype(float) #change
new_df['SPXTR'] = new_df['SPXTR'].astype(float)

yr_periods = 252

# Format for book display
font = {'family' : 'DejaVu Sans',
        'weight' : 'normal',
        'size'   : 8}
matplotlib.rc('font', **font)

def equity_graph(new_df):
    new_df = new_df / new_df.iloc[0]

    # Calculate correlation
    new_df['Correlation'] = new_df[strat].pct_change().rolling(window=int(yr_periods / 1)).corr(new_df[bm].pct_change())    
    new_df['Drawdown'] = (new_df[strat] / new_df[strat].cummax()) - 1
    
    fig = plt.figure(figsize=(15,15))

    # First chart
    ax = fig.add_subplot(311)
    ax.set_title('Strategy Comparisons')
    ax.semilogy(new_df[strat], '-',label=strat_name, color='red', linewidth=1.0)
    ax.semilogy(new_df[bm], '-',label='SPXTR', color='grey', linewidth=1.0)
    spacing = 500
    visible = ax.xaxis.get_ticklabels()[::spacing]
    for label in ax.xaxis.get_ticklabels():
        if label not in visible:
            label.set_visible(False)
    ax.set_ylabel('Returns')
    ax.set_xlabel('Duration')
    plt.xticks(rotation=None)
    ax.legend()
    
    # Second chart
    ax = fig.add_subplot(312)
    ax.fill_between(new_df.index, new_df['Drawdown'], label='Drawdown', color='blue',linewidth=1.0)
    spacing = 500
    visible = ax.xaxis.get_ticklabels()[::spacing]
    for label in ax.xaxis.get_ticklabels():
        if label not in visible:
            label.set_visible(False)
    plt.xticks(rotation=None)
    ax.legend()

    # Third chart
    ax = fig.add_subplot(313)
    ax.fill_between(new_df.index, new_df['Correlation'], label='6M Rolling Correlation', color='teal', linewidth=1.0)
    spacing = 500
    visible = ax.xaxis.get_ticklabels()[::spacing]
    for label in ax.xaxis.get_ticklabels():
        if label not in visible:
            label.set_visible(False)
    plt.xticks(rotation=None)
    ax.legend()

equity_graph(new_df)

strat = 'counter_trend'
strat_name = strat_names[strat]

df = pd.read_csv(base_path + strat + '.csv', index_col=0, parse_dates=True, names=[strat] )

monthly_data = em.aggregate_returns(df[strat].pct_change(),'monthly')
yearly_data = em.aggregate_returns(df[strat].pct_change(),'yearly')

# Start off an HTML table for display 
table = """
<table id='monthlyTable' class='table table-hover table-condensed table-striped'>

<th style="text-align:top">Year</th>
<th style="text-align:top">Jan</th>
<th style="text-align:top">Feb</th>
<th style="text-align:top">Mar</th>
<th style="text-align:top">Apr</th>
<th style="text-align:top">May</th>
<th style="text-align:top">Jun</th>
<th style="text-align:top">Jul</th>
<th style="text-align:top">Aug</th>
<th style="text-align:top">Sep</th>
<th style="text-align:top">Oct</th>
<th style="text-align:top">Nov</th>
<th style="text-align:top">Dec</th>
<th style="text-align:top">Year</th>

<tr>"""

first_year = True
first_month = True
yr = 0
mnth = 0
for m, val in monthly_data.iteritems():
    yr = m[0]
    mnth = m[1]

    if(first_month):
        table += "<td align='right'><b>{}</b></td>\n".format(yr)
        first_month = False

    if(first_year): # pad empty months for first year if sim doesn't start in January
        first_year = False
        if(mnth > 1):
            for i in range(1, mnth):
                table += "<td align='right'>-</td>\n"

    table += "<td align='right'>{:+.1f}</td>\n".format(val * 100)

    if(mnth==12): # check for dec, add yearly
        table += "<td align='right'><b>{:+.1f}</b></td>\n".format(yearly_data[yr] * 100)     
        table += '</tr>\n <tr> \n'    
        first_month = True

# add padding for empty months and last year's value
if(mnth != 12):
    for i in range(mnth+1, 13):
        table += "<td align='right'>-</td>\n"
        if(i==12):
            table += "<td align='right'><b>{:+.1f}</b></td>\n".format(
                yearly_data[yr] * 100
            ) 
            table += '</tr>\n <tr> \n'
table += '</tr>\n </tbody> \n </table>'

display(HTML(table))

def holding_period_map(new_df):
    yr = em.aggregate_returns(new_df[strat].pct_change(), 'yearly')
    new_df = pd.DataFrame(columns=range(1,len(yr)+1), index=yr.index)

    yr_start = 0
    
    table = "<table class='table table-hover table-condensed table-striped'>"
    table += "<tr><th>Years</th>"
    
    for i in range(len(yr)):
        table += "<th>{}</th>".format(i+1)
    table += "</tr>"

    for the_year, value in yr.iteritems(): # Iterates years
        table += "<tr><th>{}</th>".format(the_year) # New table row
        
        for yrs_held in (range(1, len(yr)+1)): # Iterates yrs held 
            if yrs_held   <= len(yr[yr_start:yr_start + yrs_held]):
                ret = em.annual_return(yr[yr_start:yr_start + yrs_held], 'yearly' )
                table += "<td>{:+.0f}</td>".format(ret * 100)
        table += "</tr>"    
        yr_start+=1
    return table

table = holding_period_map(df)
display(HTML(table))


In [ ]:
new_df

In [ ]:
new_df['dow_model'] = new_df['dow_model'].astype(float)
new_df['SPXTR'] = new_df['SPXTR'].astype(float)

# 💲**DIGITAL ASSETS PROGRAMME**



## Loading and manipulating data


In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from datetime import datetime as dt
from scipy import stats
import seaborn as sn
import matplotlib.pyplot as plt


In [5]:
A = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/random_stocks/APA.csv', parse_dates=True, index_col=0)
print(A.head(5))
A.info()

                open      high       low     close      volume  dividend  \
trade_date                                                                 
1996-01-02  1.689357  1.689357  1.689357  1.689357   458625.36       0.0   
1996-01-03  1.667902  1.667902  1.667902  1.667902   982282.90       0.0   
1996-01-04  1.665067  1.665067  1.665067  1.665067  1076858.16       0.0   
1996-01-05  1.620609  1.620609  1.620609  1.620609   594793.50       0.0   
1996-01-08  1.651401  1.651401  1.651401  1.651401   827370.88       0.0   

            in_sp500  
trade_date            
1996-01-02         0  
1996-01-03         0  
1996-01-04         0  
1996-01-05         0  
1996-01-08         0  
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5790 entries, 1996-01-02 to 2018-12-31
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   open      5790 non-null   float64
 1   high      5790 non-null   float64
 2   low       5790 non-nu

In [6]:
A2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/random_stocks/APA.csv')
print(A.head(5))
A2.info()

                open      high       low     close      volume  dividend  \
trade_date                                                                 
1996-01-02  1.689357  1.689357  1.689357  1.689357   458625.36       0.0   
1996-01-03  1.667902  1.667902  1.667902  1.667902   982282.90       0.0   
1996-01-04  1.665067  1.665067  1.665067  1.665067  1076858.16       0.0   
1996-01-05  1.620609  1.620609  1.620609  1.620609   594793.50       0.0   
1996-01-08  1.651401  1.651401  1.651401  1.651401   827370.88       0.0   

            in_sp500  
trade_date            
1996-01-02         0  
1996-01-03         0  
1996-01-04         0  
1996-01-05         0  
1996-01-08         0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5790 entries, 0 to 5789
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   trade_date  5790 non-null   object 
 1   open        5790 non-null   float64
 2   high        5790 non-null   flo

In [ ]:
BTC = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/crypto/crypto_bundl/BTC.csv',parse_dates=True, index_col=0)

print(BTC.head(5))
BTC.info()

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/crypto_data/coin_Aave_1.csv', parse_dates=True, index_col=0)

#Set date fomat 'dd-mm-yy'
'''
df['Date']= pd.to_datetime(df['Date']).dt.strftime('%d-%m-%y')
'''
#Drop columns not needed
df= df.drop(columns =['Name', 'Symbol','Marketcap'])

# Rearrange columns
df = df[['Date','Open','High','Low','Close','Volume']]

# Rename columns
df.rename(columns = {'Date':'trade_date','Open':'open', 'High':'high',
                     'Low':'low','Close':'close',
                     'Volume':'volume'}, inplace = True)

# reset 'trade_date' column to index
df.set_index("trade_date", inplace = True)


print(df)
df.info()


In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/crypto/AAVE_1.csv')

## test

In [ ]:
#@title
import pandas as pd
from datetime import datetime
import numpy as np


range_data = pd.date_range(start ='1/1/2010', end ='24/05/2022',
                                                  freq ='D')
 
random_df = pd.DataFrame(range_data, columns =['date'])
random_df['high'] = np.random.randint(0, 100, size =(len(range_data)))
random_df['open'] = np.random.randint(0, 100, size =(len(range_data)))
random_df['low'] = np.random.randint(0, 100, size =(len(range_data)))
random_df['close'] = np.random.randint(0, 100, size =(len(range_data)))
random_df['volume'] = np.random.randint(0, 100, size =(len(range_data)))

 
random_df['trade_date'] = pd.to_datetime(random_df['date'])
random_df = random_df.set_index('trade_date')
random_df.drop(['date'], axis = 1, inplace = True)


for col in random_df.columns:
    if np.issubdtype(random_df[col].dtype, np.number):
        random_df[col].values[:] = 100

random_df


In [ ]:
import exchange_calendars as xcals

## Adjusting timeseries data for NYSE (US) Calender

In [7]:
#import required libraries
import pandas as pd
from datetime import datetime
import exchange_calendars as xcals
from zipline import get_calendar
import numpy as np

#load raw data from file
raw_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/crypto/coinmetrics_data/btc.csv', parse_dates=True, index_col=0)

#drop columns not needed
raw_data.drop(raw_data.iloc[:, 0:67], inplace = True, axis = 1)


#drop columns not needed. Only closing prices required
raw_data.drop(raw_data.iloc[:, 1:76], inplace = True, axis = 1)

#Add new columns
raw_data['high'] =0
raw_data['low'] =0
raw_data['open'] =0
raw_data['volume'] =0

# Rearrange columns to set timestamp as first column
raw_data = raw_data[['high','open','low','PriceUSD','volume']]

#Change 'PriceUSD' columns to 'close'
raw_data.rename(columns = {'PriceUSD':'close'}, inplace = True)

#Drop row (date) not needed
raw_data=raw_data.drop(['2022-05-25'])


#Fill 'close' column with integer values '0'
raw_data["close"].fillna(0, inplace=True)

#Reset index
raw_data.reset_index(inplace = True)

#Rename time columns to date
raw_data = raw_data.rename(columns={'time': 'date'}, index=None)

#Drop all NaN values from dataframe
raw_data.dropna()

#Convert columns datatype from integer to float
raw_data["high"] = raw_data["high"].astype(float)
raw_data["open"] = raw_data["open"].astype(float)
raw_data["low"] = raw_data["low"].astype(float)
raw_data["volume"] = raw_data["volume"].astype(float)



#Confirm infrormation of dataframe
raw_data.info()

#print dataframe
raw_data


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4890 entries, 0 to 4889
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    4890 non-null   datetime64[ns]
 1   high    4890 non-null   float64       
 2   open    4890 non-null   float64       
 3   low     4890 non-null   float64       
 4   close   4890 non-null   float64       
 5   volume  4890 non-null   float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 229.3 KB


,date,high,open,low,close,volume
0,2009-01-03,0.0,0.0,0.0,0.000000,0.0
1,2009-01-04,0.0,0.0,0.0,0.000000,0.0
2,2009-01-05,0.0,0.0,0.0,0.000000,0.0
3,2009-01-06,0.0,0.0,0.0,0.000000,0.0
4,2009-01-07,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...
4885,2022-05-20,0.0,0.0,0.0,29224.245998,0.0
4886,2022-05-21,0.0,0.0,0.0,29420.078286,0.0
4887,2022-05-22,0.0,0.0,0.0,30331.875142,0.0
4888,2022-05-23,0.0,0.0,0.0,29088.251542,0.0


In [ ]:
#@title
# Count total null values in dataframe
print("Total Number of null values in the DataFrame : " +
	str(raw_data.isnull().sum().sum()))


In [ ]:
#@title
#@ show number of null for each column
display(raw_data.isnull().sum())


In [ ]:
#@title
#display null values
raw_data[raw_data['close'].isnull()]

In [8]:
# convert time_date col to datetime64 dtype
raw_data['date'] = pd.to_datetime(raw_data['date'], utc=True)

raw_data.set_index('date', inplace=True)

print(raw_data.index.date)


[datetime.date(2009, 1, 3) datetime.date(2009, 1, 4)
 datetime.date(2009, 1, 5) ... datetime.date(2022, 5, 22)
 datetime.date(2022, 5, 23) datetime.date(2022, 5, 24)]


In [9]:
from zipline import get_calendar

# Get all expected trading sessions in this range and reindex.
sessions = get_calendar('NYSE').sessions_in_range('2009-01-05', '2022-05-24')

btc = raw_data.reindex(sessions)

btc.info()

btc

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3371 entries, 2009-01-05 00:00:00+00:00 to 2022-05-24 00:00:00+00:00
Freq: C
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   high    3371 non-null   float64
 1   open    3371 non-null   float64
 2   low     3371 non-null   float64
 3   close   3371 non-null   float64
 4   volume  3371 non-null   float64
dtypes: float64(5)
memory usage: 158.0 KB


,high,open,low,close,volume
2009-01-05 00:00:00+00:00,0.0,0.0,0.0,0.000000,0.0
2009-01-06 00:00:00+00:00,0.0,0.0,0.0,0.000000,0.0
2009-01-07 00:00:00+00:00,0.0,0.0,0.0,0.000000,0.0
2009-01-08 00:00:00+00:00,0.0,0.0,0.0,0.000000,0.0
2009-01-09 00:00:00+00:00,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...
2022-05-18 00:00:00+00:00,0.0,0.0,0.0,28771.220479,0.0
2022-05-19 00:00:00+00:00,0.0,0.0,0.0,30257.885248,0.0
2022-05-20 00:00:00+00:00,0.0,0.0,0.0,29224.245998,0.0
2022-05-23 00:00:00+00:00,0.0,0.0,0.0,29088.251542,0.0


In [10]:
#Reset index
btc.reset_index(inplace = True)

#Rename time columns to date
btc = btc.rename(columns={'index': 'date'}, index=None)

btc['date'].astype(np.int64)

btc.set_index('date', inplace=True)

btc.info

btc

,high,open,low,close,volume
date,,,,,
2009-01-05 00:00:00+00:00,0.0,0.0,0.0,0.000000,0.0
2009-01-06 00:00:00+00:00,0.0,0.0,0.0,0.000000,0.0
2009-01-07 00:00:00+00:00,0.0,0.0,0.0,0.000000,0.0
2009-01-08 00:00:00+00:00,0.0,0.0,0.0,0.000000,0.0
2009-01-09 00:00:00+00:00,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...
2022-05-18 00:00:00+00:00,0.0,0.0,0.0,28771.220479,0.0
2022-05-19 00:00:00+00:00,0.0,0.0,0.0,30257.885248,0.0
2022-05-20 00:00:00+00:00,0.0,0.0,0.0,29224.245998,0.0


In [11]:
#Save dataframe to csv
btc.to_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/crypto/crypto bundle/BITCOIN.csv')

In [36]:
#Load final data to file to dataframe
BITCOIN = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/crypto/crypto bundle/BITCOIN.csv')

BITCOIN.info()

BITCOIN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3371 entries, 0 to 3370
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    3371 non-null   object 
 1   high    3371 non-null   float64
 2   open    3371 non-null   float64
 3   low     3371 non-null   float64
 4   close   3371 non-null   float64
 5   volume  3371 non-null   float64
dtypes: float64(5), object(1)
memory usage: 158.1+ KB


,date,high,open,low,close,volume
0,2009-01-05 00:00:00+00:00,0.0,0.0,0.0,0.000000,0.0
1,2009-01-06 00:00:00+00:00,0.0,0.0,0.0,0.000000,0.0
2,2009-01-07 00:00:00+00:00,0.0,0.0,0.0,0.000000,0.0
3,2009-01-08 00:00:00+00:00,0.0,0.0,0.0,0.000000,0.0
4,2009-01-09 00:00:00+00:00,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...
3366,2022-05-18 00:00:00+00:00,0.0,0.0,0.0,28771.220479,0.0
3367,2022-05-19 00:00:00+00:00,0.0,0.0,0.0,30257.885248,0.0
3368,2022-05-20 00:00:00+00:00,0.0,0.0,0.0,29224.245998,0.0
3369,2022-05-23 00:00:00+00:00,0.0,0.0,0.0,29088.251542,0.0


In [37]:
BITCOIN['date']= pd.to_datetime(BITCOIN['date']).dt.strftime('%d-%m-%y')


BITCOIN

,date,high,open,low,close,volume
0,05-01-09,0.0,0.0,0.0,0.000000,0.0
1,06-01-09,0.0,0.0,0.0,0.000000,0.0
2,07-01-09,0.0,0.0,0.0,0.000000,0.0
3,08-01-09,0.0,0.0,0.0,0.000000,0.0
4,09-01-09,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...
3366,18-05-22,0.0,0.0,0.0,28771.220479,0.0
3367,19-05-22,0.0,0.0,0.0,30257.885248,0.0
3368,20-05-22,0.0,0.0,0.0,29224.245998,0.0
3369,23-05-22,0.0,0.0,0.0,29088.251542,0.0


In [38]:

BITCOIN['date'].astype('int64')

BITCOIN

ValueError: ignored

## Removing weekends from timeseries

In [ ]:
#@title
#Import statements
%matplotlib inline
import pandas as pd
import datetime as dt
import numpy as np

#Load csv file from disc
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/crypto/crypto_data/DOGE.csv')

#Set date column to datetime
df['time'] = pd.to_datetime(df['time'], errors='coerce')

#Remove weekends from data
df = df[df.time.dt.weekday < 5]

#Set 'date' columns as Index
df.set_index("time", inplace = True)

#display weekday dataframe
df


In [ ]:
#inspect dataframe
df.info()

In [ ]:
#Save dataframe to csv

df.to_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/crypto/crypto weekday data/DODGE_weekday.csv')

In [ ]:
#Load final data to file to dataframe
dodge = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/crypto/crypto weekday data/DODGE_weekday.csv')

print(dodge)



In [ ]:
#Remove weekends from dataframe
btc = usholidays[usholidays.date.dt.weekday < 5]

#Set 'date' column as index
btc.set_index('date', inplace = True)

btc

In [ ]:
#import US holiday calender
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

#Set period between start and end date in dataframe to identify and remove holidays
holidays = calendar().holidays(start='2009-01-05', end='2022-05-24') 
m = raw_data['date'].isin(holidays)
usholidays = raw_data[~m].copy()

#print new dataframe with holidays removed
usholidays

## Create Timeseries with random values

In [ ]:
#Import liabraries
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

#Set variable for US holidays
usb = CustomBusinessDay(calendar = USFederalHolidayCalendar())

#Period between start and end date in data to remove non business days
rng = pd.date_range('2009-01-05', '2022-05-24', freq=usb)

#Print timeseries
rng

In [ ]:
#Set index as 'date' column
df1 = rng.to_frame(index = True)

#Add OHLC and Volume columns and fill with integer values 
df1['high'] = 0
df1['open'] = 0
df1['low'] = 0
df1['close'] = 0
df1['volume'] = 0

#Drop date column
df1.drop(df1.columns[[0]], axis = 1, inplace = True)


#Reset index and pass data2 variable as new dataframe
data2 = df1.reset_index()


#Rename date column from 'time' to 'date'
df_new = data2.rename(columns={'index': 'date'}, index=None)

print(df_new.head(5))
df_new.info()

In [ ]:
#save dataframe to csv. File name is the date and time it was saved/created
df_new.to_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/crypto/Random Data/300620222306.csv')

In [ ]:
#Load csv. Variable name is the date and time the csv file was loaded
dodge = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/crypto/crypto weekday data/DODGE_weekday.csv')

print(dodge)
test300620221107 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/crypto/crypto weekday data/DODGE_weekday.csv')

print(test300620221107)